In [1]:
from __future__ import print_function
import os
import neat

import pandas as pd
import numpy as np
import random

import torch
import torch.nn as nn
import torch.optim as optim


from explaneat.core.backprop import NeatNet
from explaneat.core import backprop
from explaneat.core.backproppop import BackpropPopulation
from explaneat.visualization import visualize

from sklearn import datasets
from sklearn import metrics
from sklearn.preprocessing import StandardScaler


In [2]:
random.seed(4242)
nGenerations = 3

In [3]:
def one_hot_encode(vals):
    width = max(vals)
    newVals = []
    for val in vals:
        blank = [0. for _ in range(width + 1)]
        blank[val] = 1.
        newVals.append(blank)
    return np.asarray(newVals)


In [4]:
iris = datasets.load_iris()
xs_raw = iris.data[:, :2]  # we only take the first two features.
scaler = StandardScaler()
scaler.fit(xs_raw)
xs = scaler.transform(xs_raw)
ys = iris.target
ys_onehot = one_hot_encode(ys)

In [5]:
def eval_genomes(genomes, config):
    loss = nn.CrossEntropyLoss()
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        preds = []
        for xi in xs:
            preds.append(net.activate(xi))
        genome.fitness = float(1./loss(torch.tensor(preds), torch.tensor(ys)))

In [10]:
config_path = "./config-iris"

In [11]:
config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)

# Create the population, which is the top-level object for a NEAT run.
p = BackpropPopulation(config, xs, ys, criterion=nn.CrossEntropyLoss())

# Add a stdout reporter to show progress in the terminal.
p.add_reporter(neat.StdOutReporter(True))
stats = neat.StatisticsReporter()
p.add_reporter(stats)
p.add_reporter(neat.Checkpointer(5))
bpReporter = backprop.BackpropReporter(True)
p.add_reporter(bpReporter)

In [ ]:
# Run for up to nGenerations generations.
winner = p.run(eval_genomes, nGenerations)


 ****** Running generation 0 ****** 



In [ ]:
p.reporters.reporters[2].save_checkpoint(p.config, p.population, p.species, str(p.generation) + "-final")  

# Display the winning genome.
print('\nBest genome:\n{!s}'.format(winner))

# Show output of the most fit genome against training data.
print('\nOutput:')
winner_net = neat.nn.FeedForwardNetwork.create(winner, config)
# for xi, xo in zip(xor_inputs, xor_outputs):
#     output = winner_net.activate(xi)
#     print("input {!r}, expected output {!r}, got {!r}".format(xi, xo, output))

results = []
for xi, xo in zip(xs, ys):
    output = winner_net.activate(xi)
    print("input {!r}, expected output {!r}, got {!r}".format(xi, xo, output))
    results.append([xi[0], xi[1], output])

df = pd.DataFrame(results)
df.to_csv('results.csv')

node_names = {-1:'A', -2: 'B', 0:'A XOR B'}
visualize.draw_net(config, winner, True)
visualize.plot_stats(stats, ylog=False, view=False)
visualize.plot_species(stats, view=False)

In [ ]:
p = run(config_path, 0)

g = p.best_genome

config = neat.Config(neat.DefaultGenome, neat.DefaultReproduction,
                     neat.DefaultSpeciesSet, neat.DefaultStagnation,
                     config_path)

net = NeatNet(g, config) 

winnerNet = neat.nn.FeedForwardNetwork.create(g, config)

ancestry = p.reporters.reporters[3].trace_ancestry_of_species(g.key, p.reproduction.ancestors) 

print('have ancestry')

ancestors = {
    k: v['genome'] for k, v in p.reporters.reporters[3].ancestry.items()
}
print('have ancestors')
visualize.create_ancestry_video(p.config, g, ancestry, ancestors, p.reporters.reporters[1])
print('have finished video')